In [1]:
# Import the dependencies.
import pandas as pd
import numpy as np
import time

In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [3]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=ed5c4a5d13114591e5ee76b3bbaf9589


In [4]:
# Create a set of random latitude and longitude combinations. zip packs the data in pairs
lats = np.random.uniform(low=-90.000, high=90.000, size=25)
lngs = np.random.uniform(low=-180.000, high=180.000, size=25)
lat_lngs = zip(lats, lngs)
lat_lngs

In [5]:
# Add the latitudes and longitudes to a list. This command is unpacking the zip array into a tuple list, 
# you can only do this once as it then removes it from memory
coordinates = list(lat_lngs)

In [6]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

24

In [7]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city. city.replace(" ","+") takes out the spaces and concatonates the name together
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_description = city_weather["weather"][0]["description"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | yatou
Processing Record 2 of Set 1 | norman wells
Processing Record 3 of Set 1 | orativ
Processing Record 4 of Set 1 | saldanha
Processing Record 5 of Set 1 | erzin
Processing Record 6 of Set 1 | chokurdakh
Processing Record 7 of Set 1 | dikson
Processing Record 8 of Set 1 | ushuaia
Processing Record 9 of Set 1 | villa carlos paz
Processing Record 10 of Set 1 | narsaq
Processing Record 11 of Set 1 | tocopilla
Processing Record 12 of Set 1 | port elizabeth
Processing Record 13 of Set 1 | bluff
Processing Record 14 of Set 1 | cienfuegos
Processing Record 15 of Set 1 | inirida
Processing Record 16 of Set 1 | vrangel
Processing Record 17 of Set 1 | aosta
Processing Record 18 of Set 1 | georgetown
Processing Record 19 of Set 1 | brigantine
Processing Record 20 of Set 1 | nikolskoye
Processing Record 21 of Set 1 | los llanos de aridane
Processing Record 22 of Set 1 | vardo
Processing Record 23 of Set 1

In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Yatou,37.1500,122.3833,48.67,39,0,15.79,CN,clear sky
1,Norman Wells,65.2820,-126.8329,17.64,45,20,9.22,CA,few clouds
2,Orativ,49.1889,29.5270,37.47,90,80,9.31,UA,broken clouds
3,Saldanha,-33.0117,17.9442,60.22,79,51,5.61,ZA,light rain
4,Erzin,36.9559,36.2000,64.74,38,100,1.70,TR,overcast clouds
5,Chokurdakh,70.6333,147.9167,-3.75,90,15,6.02,RU,few clouds
6,Dikson,73.5069,80.5464,-0.18,80,14,19.24,RU,few clouds
7,Ushuaia,-54.8000,-68.3000,37.06,81,40,25.32,AR,scattered clouds
8,Villa Carlos Paz,-31.4241,-64.4978,65.39,55,0,4.00,AR,clear sky
9,Narsaq,60.9167,-46.0500,32.22,83,24,0.76,GL,few clouds


In [9]:
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp","Humidity","Cloudiness","Wind Speed","Current Description"]
city_data_df=city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Yatou,CN,37.1500,122.3833,48.67,39,0,15.79,clear sky
1,Norman Wells,CA,65.2820,-126.8329,17.64,45,20,9.22,few clouds
2,Orativ,UA,49.1889,29.5270,37.47,90,80,9.31,broken clouds
3,Saldanha,ZA,-33.0117,17.9442,60.22,79,51,5.61,light rain
4,Erzin,TR,36.9559,36.2000,64.74,38,100,1.70,overcast clouds
5,Chokurdakh,RU,70.6333,147.9167,-3.75,90,15,6.02,few clouds
6,Dikson,RU,73.5069,80.5464,-0.18,80,14,19.24,few clouds
7,Ushuaia,AR,-54.8000,-68.3000,37.06,81,40,25.32,scattered clouds
8,Villa Carlos Paz,AR,-31.4241,-64.4978,65.39,55,0,4.00,clear sky
9,Narsaq,GL,60.9167,-46.0500,32.22,83,24,0.76,few clouds


In [12]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Databse.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")